# Agents II - Reflection

* Refelction:  Taking another page out of the repertoire of human thought patterns,
 this is about giving your LLM app the opportunity to analyze its past
 output and choices, together with the ability to remember reflections
 from past iterations.

In [1]:
from typing import Annotated, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
)
from langchain_ollama import ChatOllama

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages


In [20]:
model = ChatOllama(model="llama3.2:1b")

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [21]:
generate_prompt = SystemMessage(
    """You are an essay assistant tasked with writing excellent 3-paragraph 
        essays."""
    "Generate the best essay possible for the user's request."
    """If the user provides critique, respond with a revised version of your 
        previous attempts."""
)

def generate(state: State) -> State:
    answer = model.invoke([generate_prompt] + state["messages"])
    return {"messages": [answer]}

In [22]:
reflection_prompt = SystemMessage(
    """You are a teacher grading an essay submission. Generate critique and 
        recommendations for the user's submission."""
    """Provide detailed recommendations, including requests for length, depth, 
        style, etc."""
)

def reflect(state: State) -> State:
    # Invert the messages to get the LLM to reflect on its own output
    cls_map = {AIMessage: HumanMessage, HumanMessage: AIMessage}
    # First message is the original user request. 
    # We hold it the same for all nodes
    translated = [reflection_prompt, state["messages"][0]] + [
        cls_map[msg.__class__](content=msg.content) 
            for msg in state["messages"][1:]
    ]
    answer = model.invoke(translated)
    # We treat the output of this as human feedback for the generator
    return {"messages": [HumanMessage(content=answer.content)]}

In [23]:
def should_continue(state: State):
    if len(state["messages"]) > 2:
        # End after 3 iterations, each with 2 messages
        return END
    else:
        return "reflect"


In [24]:
builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("reflect", reflect)
builder.add_edge(START, "generate")
builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")

graph = builder.compile()

In [25]:
input = {
  "messages": [
    HumanMessage("""Write about The Little Prince book""")
  ]
}
for c in graph.stream(input):
    print(c)

{'generate': {'messages': [AIMessage(content='Here is a 3-paragraph essay about "The Little Prince":\n\n"The Little Prince" is a timeless novella written by Antoine de Saint-Exupéry, first published in 1943. The story follows the journey of a young prince who travels from his own asteroid to various other planets, meeting strange characters along the way. Through his adventures, the little prince learns valuable lessons about love, friendship, and the importance of human connections.\n\nAs the little prince navigates through space, he encounters a diverse array of inhabitants, each with their unique characteristics and perspectives. There\'s the rose who represents the beauty of love; the fox who symbolizes cunning and adaptability; and the king who embodies the power of authority. However, as the story progresses, it becomes clear that these characters are not what they seem, and that the little prince must confront the harsh realities of life on his own planet. Through this journey, 